In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, transforms


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Mean and std for FashionMNIST
])

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = transform

)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = transform
)



In [9]:
# wrap in dataloader
batch_size = 64
training_dataloader = DataLoader(training_data, batch_size = batch_size)
testing_dataloader = DataLoader(test_data, batch_size = batch_size)

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Maintains spatial dimensions
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 28x28 -> 14x14
            
            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14x14 -> 7x7
            
            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 7x7 -> 3x3
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Add regularisation
            nn.Linear(256, 10)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1152, out_features=256, bias=True)
    (2): ReLU()
    (3): Dr

In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [38]:
def train(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

        if batch % 100 == 0:

            loss, current = loss.item(), (batch+1)*len(X) # loss.item() - raw tensor loss value
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [39]:
def test(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # done for evaluation
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y, in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # number of correct prediciton in the batch compared to the total count
    test_loss /= num_batches # then gives average loss across all batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [40]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------")
    train(model, training_dataloader, loss_fn, optimizer)
    test(model, testing_dataloader, loss_fn)
print("Done!")

Epoch 1
--------------
loss: 2.466897 [   64/60000]
loss: 1.921513 [ 6464/60000]
loss: 1.873574 [12864/60000]
loss: 1.844572 [19264/60000]
loss: 1.828825 [25664/60000]
loss: 1.927642 [32064/60000]
loss: 1.749420 [38464/60000]
loss: 1.871479 [44864/60000]
loss: 1.890172 [51264/60000]
loss: 1.829253 [57664/60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.837147 

Epoch 2
--------------
loss: 1.792484 [   64/60000]
loss: 1.887702 [ 6464/60000]
loss: 1.843359 [12864/60000]
loss: 1.915522 [19264/60000]
loss: 1.798979 [25664/60000]
loss: 1.955489 [32064/60000]
loss: 1.805483 [38464/60000]
loss: 1.834171 [44864/60000]
loss: 1.857347 [51264/60000]
loss: 1.886288 [57664/60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.837147 

Epoch 3
--------------
loss: 1.838234 [   64/60000]
loss: 1.898856 [ 6464/60000]
loss: 1.900821 [12864/60000]
loss: 1.853080 [19264/60000]
loss: 1.805328 [25664/60000]
loss: 1.887508 [32064/60000]
loss: 1.794620 [38464/60000]
loss: 1.897269 [44864/60000]
loss: 1.881549 